# Global variable

Ce n'est pas idéal, mais c'est une solution temporaire pour choisir tout les paramètres au début de l'analyse sans avoir à les chercher dans les différentes cellules...

In [1]:
directory = "/media/admin-suz/Datas/Comparative-analysis_FoldsLeg/fold-5/1160"
# directory = "/home/admin-suz/Bureau/Test_seuil"

SKELETON_NAME     = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_002-GB1-fold5.fits_c700.up.NDskl.BRK.S006.a.NDskl"
IMAGE_NAME        = "20210630_EcadGFP_FastAiSc-1160x1160_px0.09um_dweel0-90_zoom2_002-GB1.tif"
# IMAGE_NAME_MYOSIN = "C1-T2-T3_leg_FastAiSc_1160x1160_px0v09-GB2.tif"
IMAGE_NAME_MYOSIN = IMAGE_NAME

# Pixel/Voxel size
# Airyscan
X_SIZE = 0.1832311/2
Y_SIZE = 0.1832311/2
Z_SIZE = 0.2201818

# Confocal
# X_SIZE = 0.2075665
# Y_SIZE = 0.2075665
# Z_SIZE = 0.4355092

In [2]:
# Save face, edge, vert dataframe from initial segmentation
SAVE_INIT_SEGMENTATION = True
# Save face, edge, vert image from initial segmentation
SAVE_INIT_SEGMENTATION_IMAGE = True

SAVE_SIGNAL_MEASURE = True

DIRECT_VISUALISATION_SKELETON = True
DIRECT_VISUALISATION_VERTEX_EDGES = True
DIRECT_VISUALISATION_FACE = True

# Analyse morphologique
MORPHOLOGICAL_ANALYSIS = True
SAVE_MORPHOLOGICAL_ANALYSIS = True
AREA = True
PERIMETER = True
NB_NEIGHBOR = True
ANISO = True
J_ORIENTATION = True
DIRECT_VISUALISATION_ORIENTATION = True
DIRECT_VISUALISATION_SIGNAL = True

# Largeur d'épaississement de la surface de la cellule
# Cette valeur est ajoutée au dessus et en dessous du plan de la face
THICKNESS = 0.5
DILATION = 3

SAVE_MORPHOLOGICAL_ANALYSIS = True
SAVE_INTENSITY_ANALYSIS = True

# Import 

In [3]:
from Dissects.io import (load_NDskl, 
                         load_image,
                         load_skeleton,
                         save_skeleton,
                         save_fits,
                         save_image,
                        )

from Dissects.image import (z_project,
                            thinning,
                            dilation)

from Dissects.geometry import Skeleton

from Dissects.draw.plt_draw import (plot_skeleton_3D,
                                    plot_face_3D,
                                    plot_junction_3D,
                                   )

from Dissects.analysis.analysis import (general_analysis,
                              cellstats
                              )

from Dissects.analysis.analysis_3D_apical import (junction_intensity,
                                                  morphology_analysis)

from Dissects.segmentation.seg_3D_apical import (generate_segmentation)


In [4]:
import os
import time

from skimage import io
import numpy as np
import pandas as pd
import copy
import sys
from skimage import morphology

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go
import pylab as pl

sys.setrecursionlimit(10000)

import plotly.express as px

from scipy.ndimage.morphology import (binary_fill_holes,
                                      binary_dilation,
                                     )
from scipy import ndimage
import scipy as sci
# %matplotlib notebook
%load_ext snakeviz

In [5]:
import matplotlib as mpl

# update fig parameter
mpl.rcParams['figure.dpi'] = 150 #300
mpl.rcParams['figure.figsize'] = (10,10)

In [6]:
# get original image
img0 = load_image(os.path.join(directory, 
                              IMAGE_NAME))
img_myosin = io.imread(os.path.join(directory, 
                              IMAGE_NAME_MYOSIN))

In [7]:
cp, fil, point, specs, cp_filament_info = load_NDskl(os.path.join(directory, SKELETON_NAME))

# #create skeleton object
skel = Skeleton(cp, fil, point, cp_filament_info, specs)


/home/admin-suz/miniconda3/lib/python3.8/site-packages/Dissects-0.0.1-py3.8.egg/Dissects/geometry/skeleton.py:25: UserWarning: As pixel can be half pixel from DisPerSE output,                   we DO NOT take the integer part of xyz values for critical points                   we DO NOT take the integer part of xyz values for filaments. 
  warnings.warn("As pixel can be half pixel from DisPerSE output, \


In [ ]:
#Clean skeleton
start = time.time()

skel.remove_lonely_cp()
skel.remove_free_filament()

end = time.time()
print(end-start)

/home/admin-suz/miniconda3/lib/python3.8/site-packages/Dissects-0.0.1-py3.8.egg/Dissects/geometry/skeleton.py:63: UserWarning: specs dictionnary is not updated, ncrit value is not updated.
  warnings.warn(


In [ ]:
skeleton_image = skel.create_binary_image()
img_binary_3d = morphology.skeletonize(skeleton_image)

In [ ]:
X_SHAPE = img0[0].shape[2]
Y_SHAPE = img0[0].shape[1]
Z_SHAPE = img0[0].shape[0]


In [ ]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    s = ndimage.generate_binary_structure(3,3)
    image_skeleton = skel.create_binary_image()
    image_skeleton = ndimage.morphology.binary_dilation(image_skeleton, structure=s)
    
    save_image(image_skeleton, 
               IMAGE_NAME[:-4] + '_output_skeleton.tif', 
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})

In [ ]:
if DIRECT_VISUALISATION_SKELETON:
    fig = plot_skeleton_3D(skel,
                          **{"x_size":X_SIZE,
                             "y_size":Y_SIZE,
                             "z_size":Z_SIZE,})
    fig.show()


# 3D vertex detection

In [ ]:
start = time.time()
face_df, edge_df, vert_df, points_df = generate_segmentation(skel, img0, **{"X_SIZE":X_SIZE,
                                                                           "Y_SIZE":Y_SIZE,
                                                                           "Z_SIZE":Z_SIZE})
end= time.time()
print(end-start)

if SAVE_INIT_SEGMENTATION : 
    vert_df.to_csv(os.path.join(directory, 'vert_df.csv'))
    edge_df.to_csv(os.path.join(directory, 'edge_df.csv'))
    face_df.to_csv(os.path.join(directory, 'face_df.csv'))


In [ ]:
if SAVE_INIT_SEGMENTATION_IMAGE :
    
    # Vertex image
    vertex_image = np.zeros([Z_SHAPE, Y_SHAPE, X_SHAPE])
    s = ndimage.generate_binary_structure(3,3)

    vertex_image[vert_df.z_pix.to_numpy().astype(int),
                 vert_df.y_pix.to_numpy().astype(int),
                 vert_df.x_pix.to_numpy().astype(int)] = 1
    # Dilate a little bit vertex to have a better viewing
    vertex_image = ndimage.morphology.binary_dilation(vertex_image, structure=s)
    
    save_image(vertex_image, 
               IMAGE_NAME[:-4] + '_output_vertex.tif', 
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})
    
    image_skeleton = skel.create_binary_image()
    image_skeleton = ndimage.morphology.binary_dilation(image_skeleton, structure=s)
    
    save_image(image_skeleton, 
               IMAGE_NAME[:-4] + '_output_skeleton.tif', 
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})

In [ ]:
from Dissects.analysis.analysis_3D_apical import face_intensity

enlarge_face = face_intensity(img_myosin, 
                              face_df, 
                              edge_df, 
                              vert_df,
                              points_df, 
                              THICKNESS, 
                              DILATION, 
                              {"X_SIZE":X_SIZE,
                               "Y_SIZE":Y_SIZE,
                               "Z_SIZE":Z_SIZE})

save_image(enlarge_face, 
           IMAGE_NAME[:-4] + '_output_enlarge_face_identity.tif', 
           directory, 
           **{"x_size":X_SIZE,
              "y_size":Y_SIZE,
              "z_size":Z_SIZE,})

In [ ]:
enlarge_face_random = np.zeros(enlarge_face.shape)
unique_value = np.unique(enlarge_face)
replace_value=[]
for i in range(len(unique_value)+1):
    rand_val = np.random.randint(10,2**8)
    while rand_val in replace_value:
        rand_val = np.random.randint(10,2**8)
    replace_value.append(rand_val)
    
for i in range(1, len(unique_value)):
    pos = np.where(enlarge_face==unique_value[i])
    enlarge_face_random[pos] = replace_value[i]

save_image(enlarge_face, 
           IMAGE_NAME[:-4] + '_output_enlarge_face_identity_random.tif', 
           directory, 
           **{"x_size":X_SIZE,
              "y_size":Y_SIZE,
              "z_size":Z_SIZE,})

In [ ]:
fsdfsdfsfsdf

In [ ]:
import random 
if DIRECT_VISUALISATION_FACE:
    fig = plot_skeleton_3D(skel,
                          **{"x_size":X_SIZE,
                             "y_size":Y_SIZE,
                             "z_size":Z_SIZE,})

    fig = plot_face_3D(skel,
                       face_df,
                       edge_df,
                       vert_df, 
                       fig,
                      **{"x_size":X_SIZE,
                         "y_size":Y_SIZE,
                         "z_size":Z_SIZE,})
    fig.show()

In [ ]:
if DIRECT_VISUALISATION_VERTEX_EDGES: 
    fig = plot_skeleton_3D(skel,
                      **{"x_size":X_SIZE,
                         "y_size":Y_SIZE,
                         "z_size":Z_SIZE,})

 
    fig = plot_junction_3D(skel,
                       face_df,
                       edge_df,
                       vert_df, 
                       fig,
                      **{"x_size":X_SIZE,
                         "y_size":Y_SIZE,
                         "z_size":Z_SIZE,})
    fig.show()
    


In [ ]:
# Tiff cell identity 
saving_image = np.zeros(enlarge_face.shape)
for e in edge_df.index:        
    saving_image[list(edge_df.loc[e, 'point_z'])[0].astype(int),
                list(edge_df.loc[e, 'point_y'])[0].astype(int),
                list(edge_df.loc[e, 'point_x'])[0].astype(int)] = e+1


save_image(saving_image, 
           IMAGE_NAME[:-4] + '_output_edge.tif', 
           directory, 
           **{"x_size":X_SIZE,
              "y_size":Y_SIZE,
              "z_size":Z_SIZE,})


In [ ]:
# Tiff edge identity aleatory
    
enlarge_edge_random = np.zeros(enlarge_face.shape)
unique_value = np.unique(edge_df.index)
replace_value = np.random.randint(1,2**16, len(unique_value))

s = sci.ndimage.generate_binary_structure(3, 3)

for e in range(len(unique_value)):  
    tmp_image = np.zeros(enlarge_face.shape)
    tmp_image[list(edge_df.loc[unique_value[e], 'point_z'])[0].astype(int),
                list(edge_df.loc[unique_value[e], 'point_y'])[0].astype(int),
                list(edge_df.loc[unique_value[e], 'point_x'])[0].astype(int)] = 1

    tmp_image = sci.ndimage.morphology.binary_dilation(
                                 tmp_image, s).astype(int)

    pos = np.where(tmp_image==1)
    enlarge_edge_random[pos] = replace_value[e]

save_image(enlarge_edge_random, 
           IMAGE_NAME[:-4] + '_output_enlarge_edge_random.tif', 
           directory, 
           **{"x_size":X_SIZE,
              "y_size":Y_SIZE,
              "z_size":Z_SIZE,})
    
    


In [ ]:
# edge_df = pd.read_csv(os.path.join(directory, 'edge_df.csv'), index_col="Unnamed: 0")
# face_df = pd.read_csv(os.path.join(directory, 'face_df.csv'), index_col="Unnamed: 0")
# vert_df = pd.read_csv(os.path.join(directory, 'vert_df.csv'), index_col="Unnamed: 0")

# Analyse

In [ ]:
# morphometry analysis

if MORPHOLOGICAL_ANALYSIS :
    morphology_analysis(face_df, 
                        edge_df, 
                        vert_df, 
                        points_df,
                        area=AREA,
                        perimeter=PERIMETER,
                        nb_neighbor=NB_NEIGHBOR,
                        aniso=ANISO,
                        j_orientation=J_ORIENTATION,
                       )
face_df.drop(-1, axis=0, inplace=True)                        

In [ ]:
if SAVE_MORPHOLOGICAL_ANALYSIS : 
    vert_df.to_csv(os.path.join(directory, 'vert_df_morphology.csv'))
    edge_df.to_csv(os.path.join(directory, 'edge_df_morphology.csv'))
    face_df.to_csv(os.path.join(directory, 'face_df_morphology.csv'))

In [ ]:
if DIRECT_VISUALISATION_ORIENTATION:
    image_skeleton = skel.create_binary_image()
    z0, y0,x0 = np.where(image_skeleton>0)
    background_skeleton = go.Scatter3d(x=x0,
                                        y=y0,
                                        z=z0,
                                        mode='markers',
                                        marker=dict(
                                            size=2,
                                            color='black',
                                            opacity=0.1
                                         )
                        )
    
    
    
    fig = go.Figure(data=[background_skeleton], )
    fig.update_layout(title='Background-Aniso', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )
    
    # Edges
    cmap = pl.cm.cividis
    max_aniso = face_df['aniso'][1:].max()
    face_df['norm_aniso'] = (face_df['aniso']/max_aniso)
    factor = 2
    startx = (face_df['fx'] - factor*face_df['norm_aniso']*face_df['orientationx'])/X_SIZE
    starty = (face_df['fy'] - factor*face_df['norm_aniso']*face_df['orientationy'])/Y_SIZE
    startz = (face_df['fz'] - factor*face_df['norm_aniso']*face_df['orientationz'])/Z_SIZE
    endx = (face_df['fx'] + factor*face_df['norm_aniso']*face_df['orientationx'])/X_SIZE
    endy = (face_df['fy'] + factor*face_df['norm_aniso']*face_df['orientationy'])/Y_SIZE
    endz = (face_df['fz'] + factor*face_df['norm_aniso']*face_df['orientationz'])/Z_SIZE
    
    for i in face_df.index[1:]:
        fig.add_trace(
                go.Scatter3d(
                    x=[startx[i], endx[i]],
                    y=[starty[i], endy[i]],
                    z=[startz[i], endz[i]],
                    mode='lines',
                    line={#"color":'red',
                          "color":'rgb'+str(cmap(int(face_df.loc[i]['norm_aniso']*cmap.N))[0:3]),
                          "width":10,
                         },                
                )
            )
        
    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)
    fig.show()
    

In [ ]:

from tifffile import tifffile

def save_image_analyse_face(enlarge_face,
                   face_df, 
                   column,
                   normalize=True,
                   normalize_max=None):
    
    if normalize:
        if normalize_max is None:
            face_df[column+'_norm'] = face_df[column]/face_df[column].max()
        else:
            face_df[column+'_norm'] = face_df[column]/normalize_max
        
    
    saving_image = np.zeros(enlarge_face.shape)
    
    for f in face_df.index:
        face_position = np.where(enlarge_face==f+1)
        if normalize:
            saving_image[face_position] = face_df.loc[f,column+'_norm']
        else:
            saving_image[face_position] = face_df.loc[f,column]
    
    save_image(saving_image, 
               IMAGE_NAME[:-4] +'_'+ column +'.tif',
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})


# Tiff cell identity 
save_image_analyse_face(enlarge_face, face_df, 'perimeter_approximate', normalize=False)

In [ ]:
def save_image_aniso(skeleton,
                    face_df,
                    edge_df,
                    vert_df,
                    normalize=True,
                    normalize_max=None,
                    factor=2):
    
    from skimage.draw import line_nd
    from scipy.ndimage import grey_dilation
    
    image_skeleton = skeleton.create_binary_image()
    image_skeleton = np.zeros(image_skeleton.shape)
    if normalize:
        if normalize_max is None:
            face_df['aniso_norm'] = face_df['aniso']/face_df['aniso'].max()
        else:
            face_df['aniso_norm'] = face_df['aniso']/normalize_max
            
    startx = ((face_df['fx'] - factor*face_df['aniso_norm']*face_df['orientationx'])/X_SIZE).to_numpy().astype(int)
    starty = ((face_df['fy'] - factor*face_df['aniso_norm']*face_df['orientationy'])/Y_SIZE).to_numpy().astype(int)
    startz = ((face_df['fz'] - factor*face_df['aniso_norm']*face_df['orientationz'])/Z_SIZE).to_numpy().astype(int)
    endx = ((face_df['fx'] + factor*face_df['aniso_norm']*face_df['orientationx'])/X_SIZE).to_numpy().astype(int)
    endy = ((face_df['fy'] + factor*face_df['aniso_norm']*face_df['orientationy'])/Y_SIZE).to_numpy().astype(int)
    endz = ((face_df['fz'] + factor*face_df['aniso_norm']*face_df['orientationz'])/Z_SIZE).to_numpy().astype(int)
    
    for i in face_df.index:
        
        tmp_image = np.zeros(image_skeleton.shape)
        coords = line_nd((startx[i-1], starty[i-1], startz[i-1]),
                         (endx[i-1], endy[i-1], endz[i-1]), 
                         endpoint=True,
                         integer=True)
        
        tmp_image[coords[2],
               coords[1], 
               coords[0]] = 1
        #enlarge
        s = sci.ndimage.generate_binary_structure(3, 3)
        
        tmp_image = sci.ndimage.morphology.binary_dilation(
                                     tmp_image, s).astype(int)
        
        pos = np.where(tmp_image==1)
        image_skeleton[pos] = face_df.loc[i, 'aniso_norm']
 

    save_image(image_skeleton, 
               IMAGE_NAME[:-4] +'_aniso_orientation.tif',
               directory, 
               **{"x_size":X_SIZE,
                  "y_size":Y_SIZE,
                  "z_size":Z_SIZE,})

In [ ]:
save_image_aniso(skel, 
                face_df, 
                edge_df,
                vert_df, 
                factor=4)

## Mesure d'intensité au niveau des jonctions

In [ ]:
start = time.time()
junction_intensity(img_myosin, edge_df, points_df, DILATION, 'myosin_intensity')
end= time.time()
print(end-start)

In [ ]:
if SAVE_SIGNAL_MEASURE:
    myosin_intensity = np.zeros(img_myosin.shape)
    for e, val in edge_df.iterrows():
        x_ = list(points_df[points_df.edge==e]['x_pix'].to_numpy())
        y_ = list(points_df[points_df.edge==e]['y_pix'].to_numpy())
        z_ = list(points_df[points_df.edge==e]['z_pix'].to_numpy())
    
        myosin_intensity[z_, y_, x_] = (val.myosin_intensity_mean/np.max(edge_df.myosin_intensity_mean)*255).astype('int')
    
    save_image(myosin_intensity, IMAGE_NAME[:-4] + '_myosin_intensity.tif', directory, **{"x_size":X_SIZE,
                                                                                        "y_size":Y_SIZE,
                                                                                        "z_size":Z_SIZE,})

In [ ]:
if SAVE_MORPHOLOGICAL_ANALYSIS : 
    edge_df.to_csv(os.path.join(directory, 'edge_df_intensity.csv'))

In [ ]:
if DIRECT_VISUALISATION_SIGNAL:
    vertex_isolated = go.Scatter3d(x=vert_df.x_pix,
                          y=vert_df.y_pix,
                          z=vert_df.z_pix,
                          mode='markers',
                         marker=dict(
                            size=2,
                            color='black', 
                            opacity=1
                         )
        )



    fig = go.Figure(data=[vertex_isolated], )
    fig.update_layout(title='Filament', 
                      autosize=False,
                      width=1000,
                      height=1000,
                      margin=dict(l=65, r=50, b=65, t=90),
                     )



    zs,ys,xs = vert_df.loc[edge_df['srce']][["z_pix", "y_pix", "x_pix"]].values.flatten(order='F').reshape(3, edge_df.shape[0])
    zt,yt,xt = vert_df.loc[edge_df['trgt']][["z_pix", "y_pix", "x_pix"]].values.flatten(order='F').reshape(3, edge_df.shape[0])
    c = (edge_df.myosin_intensity_mean/np.max(edge_df.myosin_intensity_mean)*255).astype('int')

    import pylab as pl
    cmap = pl.cm.cividis


    for i in range(len(zs)):

        fig.add_trace(
                go.Scatter3d(
                    x=[xs[i], xt[i]],
                    y=[ys[i], yt[i]],
                    z=[zs[i], zt[i]],
                    mode='lines',
                    line={"color":[cmap(c[i]), cmap(c[i])],
                          "width":10,
                         },                
                )
            )


    fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
    fig.show()

In [ ]:
if DIRECT_VISUALISATION_SIGNAL:
    vertex_isolated = go.Scatter3d(x=vert_df.x_pix,
                          y=vert_df.y_pix,
                          z=vert_df.z_pix,
                          mode='markers',
                         marker=dict(
                            size=2,
                            color='black', 
                            opacity=1
                         )
        )



    fig = go.Figure(data=[vertex_isolated], )
    fig.update_layout(title='Filament', 
                      autosize=False,
                      width=1000,
                      height=1000,
                      margin=dict(l=65, r=50, b=65, t=90),
                     )


    import pylab as pl
    cmap = pl.cm.cividis
    c = (edge_df['myosin_intensity_mean']/np.max(edge_df['myosin_intensity_mean'])*cmap.N).astype('int')

    for e in edge_df.index:

        fig.add_trace(
        go.Scatter3d(
            x = points_df[points_df.edge==e]['x'].to_numpy()/X_SIZE,
            y = points_df[points_df.edge==e]['y'].to_numpy()/Y_SIZE,
            z = points_df[points_df.edge==e]['z'].to_numpy()/Z_SIZE,
            mode='markers',
                   marker=dict(
                    size=2,
                    color='rgb'+str(cmap(c[e])[:3]), 
                    opacity=1
                   )
        )
    )


    fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
    fig.show()

# Analyse au niveau de la face

In [ ]:
from Dissects.analysis.analysis_3D_apical import face_intensity, compute_normal, update_geom, enlarge_face_plane
from skimage.draw import polygon

In [ ]:

enlarge_face = face_intensity(img_myosin, face_df, edge_df, vert_df,points_df, THICKNESS, DILATION, {"X_SIZE":X_SIZE,
                                                                           "Y_SIZE":Y_SIZE,
                                                                           "Z_SIZE":Z_SIZE})


In [ ]:
if SAVE_MORPHOLOGICAL_ANALYSIS : 
    face_df.to_csv(os.path.join(directory, 'face_df_intensity.csv'))

In [ ]:
ssdfsdfsdf

In [ ]:
# plot prot intensity

import plotly.graph_objects as go

z0, y0,x0 = np.where(image_skeleton==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=0.5
                     )
    )


zz0, yy0,xx0 = np.where(enlarge_face>1)
fond_enlarge = go.Scatter3d(x=xx0,
                      y=yy0,
                      z=zz0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='yellow',
                        opacity=1,
                     )
    )



fig = go.Figure(data=[fond, fond_enlarge], )
fig.update_layout(
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)),
                     showlegend=False)


fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
# plot area

import plotly.graph_objects as go

import pylab as pl
cmap = pl.cm.cividis
face_df['intensity']-=face_df.intensity.min()
face_df['intensity_norm']=face_df.intensity.values/np.max(face_df.intensity.values)*255
plot_face=[]
for f in (np.unique(edge_df.face)[1:]):
#     print('rgb'+str(cmap(int(ar[f]))))
    edges= edge_df[edge_df.face==f]
    nb_vert = len(edges)
    vert_order = [edges.iloc[0].srce]
    for i in range(nb_vert):
        vert_order.append(edges[edges.trgt==vert_order[-1]]['srce'].to_numpy()[0])
    zs=vert_df.loc[vert_order].z.to_numpy()
    ys=vert_df.loc[vert_order].y.to_numpy()
    xs=vert_df.loc[vert_order].x.to_numpy()
    plot_face.append(dict(
        type='scatter3d',
        mode='lines',
        x=xs,
        y=ys,
        z=zs,
        name='',
        surfaceaxis=2, # add a surface axis ('1' refers to axes[1] i.e. the y-axis)
        surfacecolor='rgb'+str(cmap(int(face_df['intensity_norm'].iloc[f-1]))[0:3]),
#             surfacecolor=fill_colors[int(ar[f]/2)],
        line=dict(
            color='black',
            width=4
        ),
    ))
   
    
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

In [ ]:
skel.point.loc[edge_df[edge_df.face == 1].points.iloc[0][0]]

In [ ]:
zz = np.empty(0)
yy = np.empty(0)
xx = np.empty(0)
for data in edge_df[edge_df.face == 24].points:
    try:
        zz = np.concatenate(
            (zz, data[0]))
        yy = np.concatenate(
            (yy, np.fromstring(data.split('[')[2].split(',')[1], sep=',')))
        xx = np.concatenate(
            (xx, np.fromstring(data.split('[')[2].split(',')[2][:-3], sep=',')))
    except Exception as ex:
        """
        correspond aux petites jonction ajouté "à la main"
        il faut retrouver les pixels pour pouvoir mesurer les jonctions...
        """
        print(ex)
        print('strop')

In [ ]:
from skimage.draw import polygon
img_face = enlarge_face_plane(img_myosin, face_df, edge_df, vert_df, 25, 0.6, {"X_SIZE":X_SIZE,
                                                                           "Y_SIZE":Y_SIZE,
                                                                           "Z_SIZE":Z_SIZE})
if DIRECT_VISUALISATION_VERTEX_EDGES: 
    
    
    # plot skeleton
    image_skeleton = skel.create_binary_image()
    z0, y0,x0 = np.where(image_skeleton>0)
    background_skeleton = go.Scatter3d(x=x0,
                                        y=y0,
                                        z=z0,
                                        mode='markers',
                                        marker=dict(
                                            size=2,
                                            color='black',
                                            opacity=0.1
                                         )
                        )
    
    
    
    z0, y0,x0 = np.where(img_face>0)
    face = go.Scatter3d(x=x0,
                        y=y0,
                        z=z0,
                        mode='markers',
                        marker=dict(
                            size=2,
                            color='blue',
                            opacity=1
                         )
                        )
    
    fig = go.Figure(data=[background_skeleton, face], )
    fig.update_layout(title='Background-vertex', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )
    
        
    fig['layout'].update(scene=dict(aspectmode="manual", 
                                aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                                 y=Y_SIZE*Y_SHAPE, 
                                                 z=Z_SIZE*Z_SHAPE)
                                   ),
                     showlegend=False)
    fig.show()

In [ ]:
face_df.loc[25]

In [ ]:
vert_df.loc[[133,441,145,667,440,444]]

In [ ]:
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap
rand = (np.random.rand(200,3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)



z0, y0,x0 = np.where(img_binary_3d==1)
fond = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='black',
                        opacity=1
                     )
    )

z0, y0,x0 = np.where(img_plane==1)
face = go.Scatter3d(x=x0,
                      y=y0,
                      z=z0,
                      mode='markers',
                     marker=dict(
                        size=2,
                        color='blue', 
                        opacity=0.2
                     )
    )


plot_face=[]
plot_face.append(fond)
plot_face.append(face)
fig = go.Figure(data=plot_face, )
fig.update_layout(title='Filament', 
                  autosize=False,
                  width=1000,
                  height=1000,
                  margin=dict(l=65, r=50, b=65, t=90),
                 )

fig['layout'].update(scene=dict(aspectmode="data"), showlegend=False)
fig.show()

# Verification area/perimeter

In [ ]:
from Dissects.analysis.analysis_3D_apical import projected_points
f = 19

fig = go.Figure([background_skeleton])


rand = (np.random.rand(500,3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)

fig.add_trace(
        go.Scatter3d(
            x=points_df[points_df.face==f]['x_pix'].to_numpy(),
            y=points_df[points_df.face==f]['y_pix'].to_numpy(),
            z=points_df[points_df.face==f]['z_pix'].to_numpy(),
            mode='markers',
                   marker=dict(
                    size=2,
                    color='red', 
                    opacity=1
                   )
        )
    )


 # Edges
zs,ys,xs = vert_df.loc[edge_df[edge_df.face==f]['srce']][['z_pix', 'y_pix', 'x_pix']].values.flatten(order='F').reshape(3,edge_df[edge_df.face==f].shape[0])
zt,yt,xt = vert_df.loc[edge_df[edge_df.face==f]['trgt']][['z_pix', 'y_pix', 'x_pix']].values.flatten(order='F').reshape(3,edge_df[edge_df.face==f].shape[0])

rand = (np.random.rand(len(zs),3)*255).astype('int')
rand[0] = 0
cmap_rand = ListedColormap(rand)
for i in range(len(zs)):
    fig.add_trace(
            go.Scatter3d(
                x=[xs[i], xt[i]],
                y=[ys[i], yt[i]],
                z=[zs[i], zt[i]],
                mode='lines',
                line={"color":'black',
                      "width":10,
                     },                
            )
        )

    


fig['layout'].update(scene=dict(aspectmode="manual", 
                            aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                             y=Y_SIZE*Y_SHAPE, 
                                             z=Z_SIZE*Z_SHAPE)),
                 showlegend=False)
fig.update_layout(title='Filament', 
                      autosize=False,
                      width=1000,
                      height=1000,
                      margin=dict(l=65, r=50, b=65, t=90),
                     )
fig.show()

In [ ]:
rot_pos = projected_points(face_df, edge_df, points_df, f)

fig = go.Figure([])

fig.add_trace(
        go.Scatter3d(
            x=rot_pos['x'].to_numpy()/X_SIZE,
            y=rot_pos['y'].to_numpy()/Y_SIZE,
            z=rot_pos['z'].to_numpy()/Z_SIZE,
            mode='markers',
                   marker=dict(
                    size=2,
                    color='blue', 
                    opacity=1
                   )
        )
    )
    
fig.add_trace(
        go.Scatter3d(
            x=rot_pos['x'].to_numpy()/X_SIZE,
            y=rot_pos['y'].to_numpy()/Y_SIZE,
            z=rot_pos['z'].to_numpy()*0,
            mode='markers',
                   marker=dict(
                    size=2,
                    color='green', 
                    opacity=1
                   )
        )
    )
fig['layout'].update(scene=dict(aspectmode="manual", 
                            aspectratio=dict(x=X_SIZE*X_SHAPE, 
                                             y=Y_SIZE*Y_SHAPE, 
                                             z=Z_SIZE*Z_SHAPE)),
                 showlegend=False)
fig.update_layout(title='Filament', 
                      autosize=False,
                      width=1000,
                      height=1000,
                      margin=dict(l=65, r=50, b=65, t=90),
                     )
fig.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(rot_pos['y'].to_numpy(),
         rot_pos['x'].to_numpy(),'.')

ax.set_aspect("equal")

In [ ]:
face_df


In [ ]:
from tyssue import Sheet
from tyssue import SheetGeometry
face_df = pd.DataFrame(index=np.arange(1, np.max(edge_df.face)))
# remove_edge = edge_df[edge_df.face==-1].index
# edge_df.drop(remove_edge, axis=0, inplace=True)
# edge_df.reset_index(inplace=True, drop=True)
data={'vert':vert_df, 'edge':edge_df, 'face':face_df}
sheet = Sheet('spherical', data)
sheet.reset_index()
sheet.reset_topo()
SheetGeometry.update_all(sheet)

from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet,list('yz'), mode='2D')

In [ ]:
from tyssue.draw import sheet_view
fig, ax = sheet_view(sheet, mode='3D')
fig